Installing the dependencies

In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

Data collection

In [4]:
products_df = pd.read_csv('products.csv')
sales_df = pd.read_csv('sales.csv')

In [5]:
products_df.head()

,sku,name,current_price,cost_price,stock
0,A123,Item A,649.99,500,150
1,B456,Item B,699.00,550,15
2,C789,Item C,999.00,500,250


In [6]:
sales_df.head()

,sku,quantity_sold
0,A123,10
1,B456,35
2,C789,0


 Merge product and sales data

In [7]:
merged_df = pd.merge(products_df, sales_df, on='sku', how='left')

Fill any missing values in quantity_sold with 0

In [8]:
merged_df['quantity_sold'] = merged_df['quantity_sold'].fillna(0)

In [9]:
# Create a new column for new prices, starting with current prices
merged_df['new_price'] = merged_df['current_price']

In [12]:
# Apply pricing rules in order of precedence
for index, row in merged_df.iterrows():
    # Rule 1: Low Stock, High Demand
    if row['stock'] < 20 and row['quantity_sold'] > 30:
        merged_df.at[index, 'new_price'] = row['current_price'] * 1.15
    
    # Rule 2: Dead Stock
    elif row['stock'] > 200 and row['quantity_sold'] == 0:
        merged_df.at[index, 'new_price'] = row['current_price'] * 0.7
    
    # Rule 3: Overstocked Inventory
    elif row['stock'] > 100 and row['quantity_sold'] < 20:
        merged_df.at[index, 'new_price'] = row['current_price'] * 0.9
    
    # Rule 4: Minimum Profit Constraint (always applied)
    min_price = row['cost_price'] * 1.2
    if merged_df.at[index, 'new_price'] < min_price:
        merged_df.at[index, 'new_price'] = min_price
    
    # Final rounding
    merged_df.at[index, 'new_price'] = round(merged_df.at[index, 'new_price'], 2)

In [13]:
 # Create output dataframe with required columns
output_df = merged_df[['sku', 'current_price', 'new_price']].copy()
output_df = output_df.rename(columns={'current_price': 'old_price'})

In [14]:
# Add $ units to prices
output_df['old_price'] = '$' + output_df['old_price'].astype(str)
output_df['new_price'] = '$' + output_df['new_price'].astype(str)

In [15]:
# Save to CSV
output_df.to_csv('updated_prices.csv', index=False)

In [16]:
# Display results
print("Updated prices calculated successfully!")
print(output_df)

Updated prices calculated successfully!
    sku old_price new_price
0  A123   $649.99    $600.0
1  B456    $699.0   $803.85
2  C789    $999.0    $699.3
